# Train MIL

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 
from pkg.wsi_mil.deepmil.train import main as train
from pkg.wsi_mil.deepmil.writes_results_cross_val import main as writes_validation_results
from pkg.wsi_mil.deepmil.writes_final_results import main as writes_test_results
import pandas as pd
import os
import datetime
import subprocess
import shutil
from argparse import ArgumentParser
import yaml

In [ ]:
out = "./outputs"
name = "test"
reps = 5
config = "/cluster/CBIO/home/ablondel1/wsi_mil/scripts/config_default.yaml"
n_ensemble = 3

In [ ]:
name = name + datetime.date.today().strftime('%Y_%m_%d')
out = os.path.abspath(out)
out = os.path.join(out, name)
config = os.path.abspath(config)
os.makedirs(out, exist_ok=True)
shutil.copy(config, os.path.join(out, 'config.yaml'))
with open(config, 'r') as f:
    dic = yaml.safe_load(f)
table = pd.read_csv(dic['table_data'])
tests = len(set(table['test'].values)) 


In [ ]:
for test in range(tests):
    for rep in range(reps):
        raw_args = [
                '--config', config, 
                '--repeat', f'{rep}', 
                '--test_fold', f'{test}', 
                ]
        wd = os.path.join(out, f'test_{test}', f'rep_{rep}')
        os.makedirs(wd, exist_ok=True)
        os.chdir(wd)
        train(raw_args=raw_args)


In [ ]:
# Root of experiment.
os.chdir(out)
raw_args = ['--n_ensemble', f'{n_ensemble}']
writes_validation_results(raw_args)
writes_test_results([])